In [3]:
# Inspired from https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.
/home/erwan/.local/lib/python3.6/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.23) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [4]:
import tensorflow as tf
with tf.Session() as sess:
  devices = sess.list_devices()
print(devices)
print('is_gpu', tf.test.is_gpu_available())
print(tf.test.gpu_device_name())

[_DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 268435456, 6366847924171642404), _DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 13451280468673011905), _DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 2292702306885670257), _DeviceAttributes(/job:localhost/replica:0/task:0/device:GPU:0, GPU, 176357376, 1310155413113992348)]
is_gpu True
/device:GPU:0


In [5]:
raw_text = open('legfrance.txt').read()
raw_text = raw_text[:5_000_000]

In [6]:
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [7]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Number of lines: ", raw_text.count('\n'))
print("Total Vocab: ", n_vocab)

Total Characters:  5000000
Number of lines:  49479
Total Vocab:  118


In [8]:
# prepare the dataset of input to output pairs encoded as integers
#seq_length = 100
seq_length = 50
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

Total Patterns:  4999950


In [9]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
#model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)
#model.fit(X, y, epochs=5, batch_size=64, callbacks=callbacks_list)

In [9]:
# load the network weights
filename = "weights-improvement-11-1.4636.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [10]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [12]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
res = ""
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    res += result
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print(res, "\nDone.")

Seed:
" ument considéré comme essentiel à l'exercice de la "
 personne dont le personnel de l'antiiteeiit de l'anticle oui le peut être peutonnée de l'antecle de l'anteileteot de l'antecle de l'anteileteot de l'antecle de l'anteileteot de l'antecle de l'anteileteot de l'antecle de l'anteileteot de l'antecle de l'anteileteot de l'antecle de l'anteileteot de l'antecle de l'anteileteot de l'antecle de l'anteileteot de l'antecle de l'anteileteot de l'antecle de l'anteileteot de l'antecle de l'anteileteot de l'antecle de l'anteileteot de l'antecle de l'anteileteot de l'antecle de l'anteileteot de l'antecle de l'anteileteot de l'antecle de l'anteileteot de l'antecle de l'anteileteot de l'antecle de l'anteileteot de l'antecle de l'anteileteot de l'antecle de l'anteileteot de l'antecle de l'anteileteot de l'antecle de l'anteileteot de l'antecle de l'anteileteot de l'antecle de l'anteileteot de l'antecle de l'anteileteot de l'antecle de l'anteileteot de l'antecle de l'anteileteot de l'antecle d

In [12]:
model.weights

[<tf.Variable 'lstm_1/kernel:0' shape=(1, 1024) dtype=float32_ref>,
 <tf.Variable 'lstm_1/recurrent_kernel:0' shape=(256, 1024) dtype=float32_ref>,
 <tf.Variable 'lstm_1/bias:0' shape=(1024,) dtype=float32_ref>,
 <tf.Variable 'dense_1/kernel:0' shape=(256, 118) dtype=float32_ref>,
 <tf.Variable 'dense_1/bias:0' shape=(118,) dtype=float32_ref>]

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 256)               264192    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 118)               30326     
Total params: 294,518
Trainable params: 294,518
Non-trainable params: 0
_________________________________________________________________


In [14]:
X.shape[1], X.shape[2]

(50, 1)

In [25]:
n = 256
m = 1
4 * ( n*m + n**2 + n)

264192

In [28]:
30326/118

257.0